In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/persona-chat/fake_dataset

Mounted at /content/drive
/content/drive/MyDrive/persona-chat/fake_dataset


In [36]:
import re

def add_apostrophes(text):
    contractions = {
        # r"\b(can)(not)\b": r"\1n\2",
        # r"\b(d)(o)\b (not)\b": r"\1\2 \3",
        # r"\b(will)(not)\b": r"\1n\2",
        # r"\b(are)\b": r"\1",
        # r"\b(is)\b": r"\1",
        # r"\b(have)\b": r"\1",
        # r"\b(has)\b": r"\1",
        # r"\b(had)\b": r"\1",
        r" (\b)(ve)\b": r"\1'\2",
        r" (\b)(s)\b": r"\1'\2",
        r" (\b)(re)\b": r"\1'\2",
        r" (\b)(t)\b": r"\1'\2",
        r" (\b)(ll)\b": r"\1'\2",
        r" (\b)(d)\b": r"\1'\2",
        r" (\b)(m)\b": r"\1'\2"
    }

    for pattern, repl in contractions.items():
        text = re.sub(pattern, repl, text, flags=re.IGNORECASE)

    return text

# text = "You must ve had your hands full. So let s talk a little bit about your duties."
# corrected_text = add_apostrophes(text)
# print(corrected_text)


You must've had your hands full. So let's talk a little bit about your duties.


In [37]:
import pandas as pd

df = pd.read_csv('fake_data.csv', delimiter=',')
json_data = []
# print(df.columns)
for _, row in df.iterrows():
    conversation_lines = row['Conversation'].split('\n')
    conversation_pairs = []
    speaker1 = row['Speaker_1']
    speaker2 = row['Speaker_2']


    for i in range(0, len(conversation_lines), 2):
        pair = {}
        if i < len(conversation_lines):
            pair[speaker1] = add_apostrophes(conversation_lines[i])
        if i + 1 < len(conversation_lines):
            pair[speaker2] = add_apostrophes(conversation_lines[i + 1])
        conversation_pairs.append(pair)

    conversation_dict = {
        'persona': row['Persona'],
        'big5': row['Big5'],
        'conversation': conversation_pairs
    }

    json_data.append(conversation_dict)

import json
with open('fake_data.json', 'w', encoding='utf-8') as file:
    json.dump(json_data, file, ensure_ascii=False, indent=4)

In [38]:
# split
from sklearn.model_selection import train_test_split

with open('fake_data.json', 'r') as file:
    data = json.load(file)

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

with open('train_fake_dataset.json', 'w') as file:
    json.dump(train_data, file)
with open('val_fake_dataset.json', 'w') as file:
    json.dump(val_data, file)